# A1111 Extension Installer (UI)

Installs **Automatic1111** extensions into the WebUI data directory so A1111 loads them correctly:

- Data root (`WEBUI_ROOT`): `/workspace/a1111` (default in your image)
- Extensions dir: `/workspace/a1111/extensions`

Re-running is safe: existing repos are updated with `git pull`. After installing, use the **Restart WebUI** cell at the bottom so changes take effect.

In [ ]:
# --- Paths / settings ---
import os
from pathlib import Path

WEBUI_ROOT = Path(os.environ.get("WEBUI_ROOT", "/workspace/a1111"))
EXT_DIR = WEBUI_ROOT / "extensions"
EXT_DIR.mkdir(parents=True, exist_ok=True)

print("WEBUI_ROOT:", WEBUI_ROOT)
print("Extensions directory:", EXT_DIR)

## Helper functions

In [ ]:
import subprocess
from urllib.parse import urlparse
from pathlib import Path

def run(cmd, cwd=None):
    print("$", " ".join(cmd))
    return subprocess.run(cmd, cwd=cwd, check=True)

def repo_basename(url: str) -> str:
    # turn https://github.com/owner/repo(.git) into "repo"
    path = urlparse(url).path.rstrip("/")
    name = path.split("/")[-1]
    return name[:-4] if name.endswith(".git") else name

def install_or_update(url: str, base: Path) -> Path:
    name = repo_basename(url)
    dst = base / name
    if not dst.exists():
        print(f"\n[install] {name} from {url}")
        run(["git", "clone", "--depth", "1", url, str(dst)])
    else:
        print(f"\n[update] {name}")
        try:
            run(["git", "-C", str(dst), "reset", "--hard"]) 
            run(["git", "-C", str(dst), "pull", "--ff-only"]) 
        except subprocess.CalledProcessError:
            print("  (non-ff; attempting rebase)")
            run(["git", "-C", str(dst), "pull", "--rebase", "--autostash"]) 
    return dst

## Pick extensions to install
Our recommended picks are pre-checked. You can also paste extra Git repo URLs at the bottom.

In [ ]:
import ipywidgets as W
from IPython.display import display, Markdown

# (label, url, default_checked)
EXT_OPTS = [
    ("Aspect Ratio",       "https://github.com/alemelis/sd-webui-ar", True),
    ("Tag Autocomplete",   "https://github.com/DominikDoom/a1111-sd-webui-tagcomplete", True),
    ("Images Browser",     "https://github.com/AlUlkesh/stable-diffusion-webui-images-browser", True),
    ("Config Presets",     "https://github.com/Zyin055/Config-Presets", True),
    ("ADetailer",          "https://github.com/Bing-su/adetailer", True),
    ("Ultimate SD Upscale","https://github.com/Coyote-A/ultimate-upscale-for-automatic1111", True),
    ("ControlNet",         "https://github.com/Mikubill/sd-webui-controlnet", False),
    ("Dynamic Prompts",    "https://github.com/adieyal/sd-dynamic-prompts", False),
    ("OpenPose Editor",    "https://github.com/fkunn1326/openpose-editor", False)
    ("Deforum",            "https://github.com/deforum/sd-webui-deforum", False)
    ("AnimateDiff",        "https://github.com/continue-revolution/sd-webui-animatediff", False)
    ("ReActor",            "https://github.com/Gourieff/sd-webui-reactor-sfw", False)
]

checks = []
url_fields = []
for label, url, checked in EXT_OPTS:
    cb = W.Checkbox(description=label, value=checked)
    tf = W.Text(value=url, layout=W.Layout(width="100%"))
    checks.append(cb)
    url_fields.append(tf)

extra_label = W.HTML("<b>Extra extension URLs (one per line)</b>")
extra_urls = W.Textarea(value="", placeholder="https://github.com/user/extension\nhttps://github.com/another/one", layout=W.Layout(width="100%", height="100px"))

btn = W.Button(description="Install/Update Selected", button_style="success", icon="download")
out = W.Output()

def on_click(_):
    out.clear_output()
    with out:
        errors = []
        installed = []

        # Selected from list
        for cb, tf in zip(checks, url_fields):
            if not cb.value:
                continue
            url = tf.value.strip()
            if not url:
                continue
            try:
                installed.append(install_or_update(url, EXT_DIR))
            except Exception as e:
                errors.append((url, str(e)))

        # Extra URLs
        ext_lines = [u.strip() for u in extra_urls.value.splitlines() if u.strip()]
        for url in ext_lines:
            try:
                installed.append(install_or_update(url, EXT_DIR))
            except Exception as e:
                errors.append((url, str(e)))

        print("\n=== Summary ===")
        for p in installed:
            print("✔", p)
        if not installed:
            print("(No changes)")
        if errors:
            print("\nErrors:")
            for url, msg in errors:
                print("✖", url, "→", msg)

display(Markdown("### Extensions"))
for cb, tf in zip(checks, url_fields):
    display(cb, tf)

display(extra_label, extra_urls)
display(btn, out)
btn.on_click(on_click)

## Restart WebUI (required after installing)
This restarts A1111 under **supervisord**. If the supervisor control socket is unavailable, it falls back to killing `launch.py` so Supervisor auto-restarts it.

In [ ]:
import time, subprocess, shlex
print("Attempting to restart the A1111 WebUI service via supervisord …")
try:
    cmd = "supervisorctl -c /etc/supervisord.conf restart a1111"
    out = subprocess.run(shlex.split(cmd), capture_output=True, text=True)
    if out.returncode == 0 and "a1111:" in (out.stdout or ""):
        print((out.stdout or out.stderr).strip() or "✔ Restarted via supervisorctl")
    else:
        print("Supervisorctl socket unavailable, forcing process restart…")
        subprocess.run("pkill -f 'launch.py'", shell=True)
        time.sleep(5)
        print("✔ WebUI process terminated, Supervisor will auto-restart it")
    print("Wait ~15 seconds, then reload your WebUI tab.")
except Exception as e:
    print("⚠️ Restart attempt failed:", e)
    print("If needed, run this in a Terminal:")
    print("supervisorctl -c /etc/supervisord.conf restart a1111")